# Read in all dataset and Data Exploration Analysis 

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
spark = SparkSession.builder.appName('EDA').getOrCreate()

In [ ]:
#%cd home/g593697882qq/archive

In [ ]:
#df = pd.read_csv("amazon_reviews_us_Baby_v1_00.tsv", sep='\t', error_bad_lines=False)
#df.shape

## Read in all 12 (21.78GB) dataset by Pyspark and form into one dataset called `data`

In [5]:
schema = StructType([ \
    StructField("marketplace",       StringType(),    True), \
    StructField("customer_id",       StringType(),    True), \
    StructField("review_id",         StringType(),    True), \
    StructField("product_id",        StringType(),    True), \
    StructField("product_parent",    StringType(),    True), \
    StructField("product_title",     StringType(),    True), \
    StructField("product_category",  StringType(),    True), \
    StructField("star_rating",       IntegerType(),   True), \
    StructField("helpful_votes",     IntegerType(),   True), \
    StructField("total_votes",       IntegerType(),   True), \
    StructField("vine",              StringType(),    True), \
    StructField("verified_purchase", StringType(),    True), \
    StructField("review_headline",   StringType(),    True), \
    StructField("review_body",       StringType(),    True), \
    StructField("review_date",       TimestampType(), True), \
  ])

path = ['archive/amazon_reviews_us_Apparel_v1_00.tsv',
        'archive/amazon_reviews_us_Automotive_v1_00.tsv',
        'archive/amazon_reviews_us_Baby_v1_00.tsv',
        'archive/amazon_reviews_us_Beauty_v1_00.tsv',
        'archive/amazon_reviews_us_Books_v1_02.tsv',
        'archive/amazon_reviews_us_Camera_v1_00.tsv',
        'archive/amazon_reviews_us_Electronics_v1_00.tsv',
        'archive/amazon_reviews_us_Furniture_v1_00.tsv',
        'archive/amazon_reviews_us_Sports_v1_00.tsv',
        'archive/amazon_reviews_us_Grocery_v1_00.tsv',
        'archive/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv',
        'archive/amazon_reviews_us_Music_v1_00.tsv']

data = spark.read.csv(path, schema=schema, header=True, sep='\t', mode='DROPMALFORMED')

In [6]:
data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



### There are in total *37,173,725* records in this huge data (big data)

In [ ]:
data.count()

In [ ]:
data.show()

## Data Exploration Analysis

### Count reviews by each product category

#### We observe that there are some dates and reviews apprearing in the product_category column.

In [ ]:
df_count = data.groupBy('product_category').count()
df_count.show(df_count.count())

#### We filter out dates and reviews in the product_category column and count each product category

In [7]:
product_category = ['Sports', 'Baby', 'Apparel', 'Grocery', 'Electronics', 'Automotive', 'Books', 'Music', 'Furniture', 'Personal_Care_Appliances', 'Camera', 'Beauty']
data_filter = data.filter(data.product_category.isin(product_category))
df_filter_count = data_filter.groupBy('product_category').count()
df_filter_count.show(df_filter_count.count())

+--------------------+-------+
|    product_category|  count|
+--------------------+-------+
|              Sports|4849563|
|                Baby|1752932|
|             Apparel|5906322|
|             Grocery|2402431|
|         Electronics|3093861|
|          Automotive|3514709|
|               Books|3105515|
|               Music|4751540|
|           Furniture| 792113|
|Personal_Care_App...|  85981|
|              Camera|1801972|
|              Beauty|5115452|
+--------------------+-------+



In [ ]:
df_filter_count.toPandas().sort_values('count', ascending=False).plot.bar(x = 'product_category', y = 'count');

### Which customers post the most reviews over this 20 years.

In [11]:
customer_count = data_filter.groupBy('customer_id').count()
customer_count.sort(col('count').desc()).show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50122160|22331|
|   50732546| 9964|
|   50736950| 7943|
|   38214553| 5412|
|   51184997| 5371|
|   18116317| 4302|
|   23267387| 4051|
|   50345651| 3842|
|   52496677| 3003|
|   14539589| 2913|
|   15725862| 2852|
|   50913245| 2851|
|   50881246| 2802|
|   20018062| 2717|
|   22073263| 2682|
|   52615377| 2678|
|   19380211| 2592|
|   51381678| 2458|
|   37455882| 2371|
|   50441674| 2366|
+-----------+-----+
only showing top 20 rows



### How many customers show up in this data?

In [12]:
customer_count.count()

13863467

In [ ]:
customer_count.toPandas().sort_values('count', ascending=False).head(20).plot.bar(x = 'customer_id', y = 'count');

## The average rating of each product category and the top 20 customers

In [10]:
data_filter.groupBy('product_category').mean('star_rating').show()

+--------------------+-----------------+
|    product_category| avg(star_rating)|
+--------------------+-----------------+
|              Sports|4.229311795722625|
|                Baby| 4.16268286505124|
|             Apparel|4.105228939431342|
|             Grocery|4.312228738307156|
|         Electronics|4.035506443243571|
|          Automotive|4.246287815008298|
|               Books|4.182722672406992|
|               Music| 4.43662139011773|
|           Furniture|4.083948881030863|
|Personal_Care_App...|3.977401984159291|
|              Camera|4.127014737187925|
|              Beauty|4.187214150382019|
+--------------------+-----------------+



In [15]:
top20 = customer_count.sort(col('count').desc()).head(20)

In [24]:
top20_l = list(map(lambda x : x[0], top20))

In [28]:
data_filter.filter(data_filter.customer_id.isin(top20_l)).groupBy('customer_id').mean('star_rating').sort(col('avg(star_rating)').desc()).show()



+-----------+------------------+
|customer_id|  avg(star_rating)|
+-----------+------------------+
|   50122160|4.9980296448882715|
|   14539589|4.9883281840027465|
|   50345651| 4.980478917230609|
|   20018062|4.9157158630842845|
|   52496677| 4.812853812853813|
|   22073263| 4.790454884414616|
|   15725862| 4.745792426367461|
|   50732546|4.6842633480529905|
|   50913245| 4.518063837250088|
|   52615377| 4.468259895444361|
|   50441674| 4.431530008453086|
|   51184997| 4.146155278346677|
|   38214553| 4.131929046563193|
|   18116317| 4.109019060901906|
|   50736950| 4.078182047085484|
|   37455882| 4.063264445381695|
|   50881246| 4.026052819414704|
|   23267387|3.9002715378918786|
|   51381678| 3.882424735557364|
|   19380211|2.9996141975308643|
+-----------+------------------+

